In [ ]:
%matplotlib inline
# Dependencies
import tweepy
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from config import (consumer_key, consumer_secret, access_token, access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [ ]:
# Getting twitter data from below news handle
News_Handle = ("BBCWorld", "CBSNews", "CNN", "Foxnews", "NYTimes")

# List for dictionaries of news sentiment result
News_Sentiment = []

for NewsOrg in News_Handle:
    
    # Variables for holding the sentiments and other variables
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []
    tweet_text = []
    created_at_ts = []
    tweet_count = []
    tweet_cnt = 1
    
    # Loop through 5 pages of tweets (total 100 tweets for each news handle) 
    for v in range(1,6):
        
        # Get all tweets from the home feed
        news_tweets = api.user_timeline(NewsOrg, page=v)
       
        # Loop through all tweets
        for tweet in news_tweets:
            
            print(f'NEWS HANDLE: {NewsOrg} TWEET#: {tweet_cnt} \nTWEET TEXT: {tweet["text"]}')
            # Running Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]
            
            # Add each value to the appropriate list
            compound_list.append(compound)
            positive_list.append(pos)
            negative_list.append(neg)
            neutral_list.append(neu)
            
            tweet_text.append(tweet["text"])
            # Getting Tweet time in raw form and converting to datetime object
            raw_time = tweet["created_at"]
            converted_time = datetime.strptime(raw_time, "%a %b %d %H:%M:%S %z %Y")
            created_at_ts.append(converted_time)
            tweet_count.append(tweet_cnt)
            
            News_Sentiment.append({"News_Org": NewsOrg,
                           "Text":tweet["text"],
                           "Created_Dt": converted_time,
                           "Compound": compound,
                           "Positive": pos,
                           "Neutral": neu,
                           "Negative": neg,
                           "Tweet_Count": tweet_cnt})
            
            tweet_cnt+=1   

In [ ]:
# Creating DataFrame 
News_Sentiment_df = pd.DataFrame(News_Sentiment)

# Reordering the columns 
News_Sentiment_rdf= News_Sentiment_df[["News_Org","Created_Dt","Text","Compound","Positive","Neutral","Negative","Tweet_Count"]]

# Saving data to CSV file
News_Sentiment_rdf.to_csv("NewsOrgs_Sentiment_Data.csv", index=False, header=True)
News_Sentiment_rdf.head()

In [ ]:
# The First Plot 
Color = {"BBCWorld":"Lightskyblue", "CBSNews":"Green", "CNN":"Red", "Foxnews":"Royalblue", "NYTimes":"Yellow"}

# Plotting scatter plot for all news orgrs in a loop
plt.figure(figsize = (7.5,5.0))
for newsorg in Color.keys():    
    Curr = News_Sentiment_rdf[News_Sentiment_rdf["News_Org"] == newsorg]
   
    # Build the scatter plots for each News Organization
    plt.scatter(Curr["Tweet_Count"], Curr["Compound"], s=100, c=Color[newsorg],edgecolors="black",label=newsorg,alpha=0.85)

# Setting Legend, Title, Labels for X & Y Axis

plt.xlabel("Tweets Ago")
plt.ylabel("Tweet Polarity")
plt.title(f'Sentiment Analysis of Media Tweets ({datetime.now().date()})')
plt.legend(title = "Media Sources", frameon = False, edgecolor = 'black', bbox_to_anchor=(1, 0.85))   

# Save Figure
plt.savefig("News_Sentiment_Analysis.png")
plt.show()

In [ ]:
# The Second Plot data setup
aggr_compound = News_Sentiment_rdf.groupby(["News_Org"]).mean()["Compound"].round(3)
print(aggr_compound)

x_axis= []
x_axis= News_Sentiment_df["News_Org"].unique() 

print(x_axis)

In [ ]:
# Plotting the graph
plt.bar(x_axis, aggr_compound, color=["Lightskyblue","Green","Red","Royalblue","Yellow"] )

# Setting value label for each bar
for i, v in enumerate(aggr_compound):
    plt.text(i-.3, v+.03, "{:+.2f}".format(v), color='black', fontweight='bold')
    
# Setting title, grid and X and Y axis labels
plt.xlabel("News Media")
plt.ylabel("Tweet Polarity")
plt.title(f'Overall Media Sentiment based on Twitter ({datetime.now().date()})')
plt.grid()

# Save Figure
plt.savefig("Overall_Twitter_News_Sentiment.png")
plt.show()